<a href="https://colab.research.google.com/github/chunam76/LLM/blob/main/%5B%EC%8B%A4%EC%8A%B5%5D_2_OpenAI_API_%EA%B8%B0%EB%B0%98_%EB%8C%93%EA%B8%80_%EB%B6%84%EB%A5%98_%EC%9E%90%EB%8F%99%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [실습] OpenAI API를 이용하여 댓글 자동 분류하기

OpenAI의 Chat API를 이용해, 댓글의 긍정/부정 여부를 분류하는 기능을 구현해 보겠습니다.   
이는 Sentiment Analysis라고 부르는 감정 분석 문제 중 하나로,   
자연어 처리에서 매우 역사가 깊은 문제입니다.   



In [ ]:
!pip install openai --upgrade

실습용 데이터는 `reviews.csv` 파일입니다.   
해당 파일을 pandas를 통해 불러옵니다.

In [ ]:
import pandas as pd

reviews = pd.read_csv('./reviews.csv')
print(reviews.shape)
reviews.head()

주어진 리뷰 데이터는 다음의 column으로 구성되어 있습니다.
- Num : 리뷰 인덱스 번호
- Review: 리뷰 텍스트
- Label: 긍정/부정 레이블 (-1:부정, 1:긍정)



In [ ]:
# os의 환경 변수에 API 키 복사 붙여넣기
import openai
import os

# OPENAI API KEY 설정
os.environ['OPENAI_API_KEY']=""

client = openai.OpenAI()


assert len(os.environ['OPENAI_API_KEY']) > 0, "OPENAI_API_KEY가 환경 변수에 설정되어 있지 않습니다. API 키를 설정해주세요."

# API 키가 설정되어 있다면, 이 지점 이후의 코드가 실행됩니다.
print("OPENAI_API_KEY가 정상적으로 설정되어 있습니다.")

가장 단순한 형태로 프롬프트를 만들어 보겠습니다.

- 지시사항이 먼저 입력되고, 줄바꿈 후 리뷰를 입력합니다.
- 리뷰의 분류 결과는 1(긍정)과 -1(부정) 중 하나로만 출력되게 합니다.

In [ ]:
system_prompt='''
주어지는 입력이 긍정/부정 중 어떤 내용을 담고 있는지 분류하세요.
긍정인 경우 '긍정', 부정인 경우 '부정'만 출력하세요.
'''

def classify_review(review, instruction):
    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = [{'role': 'system', 'content': instruction},
                  {'role': 'user', 'content': review}],
        max_tokens = 300,
        temperature = 0
        )
    return response.choices[0].message.content

In [ ]:
correct = 0
incorrect = 0
for idx, review, label in zip(reviews['Num'],reviews['Review'], reviews['Label']):
    print(f'#{idx} : ({correct}/{incorrect})')
    print(review)
    response = classify_review(review, system_prompt)
    print(response, '/', label)
    if label == -1:
        if response=='부정':
            correct+=1
        else:
            incorrect += 1
            print('Misclassification!')
    else:
        if response=='긍정':
            correct+=1
        else:
            incorrect += 1
            print('Misclassification!')
    print()
print('Correct:', correct)
print('Incorrect:', incorrect)
print('Accuracy:', correct / (correct + incorrect))


전반적으로 잘 맞추지만, 가끔 틀리기도 합니다.    
앞에서 배운 프롬프트 엔지니어링을 고려하면, 성능을 올릴 수 있을까요?

In [ ]:
good_system_prompt='''
음식점에 대한 리뷰가 주어지면, 이를 해석하고 '긍정/부정'중 하나로 최종 분류 결과를 출력하세요.

답변의 마지막에 분류 결과: 부정, 혹은 분류 결과: 긍정 을 출력하면 됩니다.
위 형식을 지키고, 분류 결과 뒤에는 별도의 내용을 출력하지 마세요.
---

'''

In [ ]:
def evaluate(system_prompt):
    correct = 0
    incorrect = 0
    for idx, review, label in zip(reviews['Num'],reviews['Review'], reviews['Label']):
        print(f'#{idx} : ({correct}/{incorrect})')
        print(review)
        response = classify_review(review, system_prompt)
        print(response, '/', label)
        if label == -1:
            if '분류 결과: 부정' in response:
                correct+=1
            else:
                incorrect += 1
                print('Misclassification!')
        else:
            if '분류 결과: 긍정' in response:
                correct+=1
            else:
                incorrect += 1
                print('Misclassification!')
        print()
    print('Correct:', correct)
    print('Incorrect:', incorrect)
    print('Accuracy:', correct / (correct + incorrect))


In [ ]:
evaluate(good_system_prompt)

<br><br><br>
프롬프트 엔지니어링에서 배운 대로, 답변을 준비할 시간을 주자 논리적 추론 성능이 향상되는 것을 볼 수 있습니다.   

성능은 올라갔지만, 아직 완벽하지 않습니다.

## [실습] 리뷰 평가 프롬프트 개선하기
주어진 프롬프트를 한번 더 개선하여, 100% 정확도의 프롬프트를 만들어 보세요!   <br><br>

In [ ]:
great_system_prompt='''

'''

In [ ]:
evaluate(great_system_prompt)

## 부록) GPT-4 VS GPT-3.5


<img src="https://github.com/NotoriousH2/img_container/assets/4037207/de39d8d4-e3bf-4a81-9a60-969fa240334f" height="80%" width="80%">
<br><br>

In [ ]:
# 참고
'''
음식점에 대한 리뷰가 주어지면, 아래의 요소에 대한 견해를 각각 30자 이내로 출력하고,
이를 바탕으로 '긍정/부정'중 하나로 최종 분류 결과를 출력하세요.
각 요소에 대한 언급이 없는 경우 생략하세요.
일반적으로, 부정에서 긍정으로 끝나는 경우 긍정 리뷰입니다.
긍정에서 부정으로 끝나는 경우 부정 리뷰입니다.
답변의 마지막에 분류 결과: 부정, 혹은 분류 결과: 긍정 을 출력하면 됩니다.
위 형식을 지키고, 분류 결과 뒤에는 별도의 내용을 출력하지 마세요.
---
1. 음식
2. 서비스
3. 가격 대비 만족도
4. 다른 식당과의 비교
5. 분위기와 음악 등
6. 분류 결과
---
'''